In [1]:
# imports
import bz2
import pickle5 as pickle
import pandas as pd

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
# functions
# Load any compressed pickle file
def decompress_pickle(file):
    data = bz2.BZ2File(file, 'rb')
    data = pickle.load(data)
    return data

In [3]:


df = decompress_pickle('6nutrients.pbz2')

df['difference'] = df.apply(lambda x: x['total.calories'] - x['goal.calories'], axis=1)

In [11]:
def score_diff(id, shiftid, diff, shiftdiff):
    if id!=shiftid or shiftid==pd.isnull:
        return -1
    elif shiftdiff>=0 and diff>=0:
        return 1
    else:
        return 0

df1 = df.copy()
# remove rows with only one day worth of data
#df1 = df.groupby('id').filter(lambda x : len(x)>1)

#df1['success'] = df1.apply(lambda x: 0 if (x['difference'] < 0 ) else 1, axis=1)

# add shiftid column
df1.insert(1, 'shiftid', df1.shift(periods=-1, axis=0)['id'])
df1['shiffdiff1'] =  df1.shift(periods=-1, axis=0)['difference']

df1['new_success'] = df1.apply(lambda x: score_diff(x['id'],x['shiftid'],x['difference'],x['shiffdiff1']), axis=1)

df1['new_success'].value_counts()

 0    460547
 1     49882
-1      8713
Name: new_success, dtype: int64

In [5]:
#